In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

from numpy.random import seed
seed(1337)
from tensorflow import random
random.set_seed(42)

import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics import classification_report, accuracy_score

import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Activation, Dense, Input, Conv1D, MaxPooling1D, Embedding, GlobalMaxPool1D, GlobalMaxPooling1D, SpatialDropout1D, LSTM, Flatten, BatchNormalization, Dropout
import transformers

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/capstone/balanced_lemmatized_cleaned_stocktwits.csv', index_col=None)

In [ ]:
data.shape

(862570, 4)

In [ ]:
data.head(10)

,created_at,body,sentiment,raw_content
14,2020-12-15T14:38:18Z,"['going', 'right', 'support', 'even', 'superio...",0,$MSFT Going right through 214 support as if it...
49,2020-12-15T14:23:04Z,"['nobody', 'gonna', 'buy', 'expensive', 'as', ...",0,$AAPL nobody gonna buy expensive ass iPhones w...
61,2020-12-15T14:12:10Z,"['robinhood', 'peep', 'gonna', 'severely', 'di...",0,$AAPL Robinhood peeps gonna be severely disapp...
103,2020-12-15T13:33:52Z,"['always', 'dump', 'dump', 'dump']",0,$AAPL always dump dump dump.
106,2020-12-15T13:30:10Z,"['turd', 'going', 'anywhere', 'pathetic']",0,$AAPL why is this turd not going anywhere. Thi...
112,2020-12-15T13:25:53Z,"['increase', 'iphone', 'sale', 'compared', 'pa...",0,$AAPL 30% increase of iPhone sales compared to...
113,2020-12-15T13:25:29Z,"['end', 'up', 'cent', 'profit', 'dry', 'paint'...",0,$AAPL Will end up 5 cents profit in this dry p...
116,2020-12-15T13:20:23Z,"['pump', 'dump']",0,$AAPL pump to dump!
210,2020-12-15T06:01:59Z,"['uh', 'oh', 'down', 'go']",0,$AAPL uh oh.. down she goes
236,2020-12-15T00:59:15Z,"['below', 'tmrw']",0,$AAPL below $120 tmrw


In [ ]:
data[data['sentiment']  == 1].shape

(431285, 4)

In [ ]:
new_bull = data[data['sentiment']  == 1].sample(300000)
new_bear = data[data['sentiment']  == 0].sample(300000)
new_df = pd.concat([new_bull, new_bear], ignore_index=True)
new_df

,created_at,body,sentiment,raw_content
0,2021-05-21T13:53:26Z,"['lucid', 'motor', 'new', 'king', 'ev', 'waiti...",1,$CCIV $TSLA Lucid Motors is the NEW King of EV...
1,2020-01-30T15:01:15Z,"['facebook', 'bargain', 'opinion']",1,$TSLA Facebook is a bargain. Opinions?
2,2020-08-18T08:17:57Z,"['love', 'tesla', 'get', 'model', 'y', 'tesla'...",1,$TSLA I love you Tesla! about to get a Model ...
3,2020-03-25T05:58:18Z,"['coronavirus', 'bill', 'passed', 'trillion', ...",1,Coronavirus bill passed.. 2 trillion into it!!...
4,2021-01-14T16:04:38Z,"['close', 'hour', 'above', 'game', 'immediate'...",1,$NVDA If this closes hour above $537.25 it&#39...
...,...,...,...,...
599995,2021-05-24T18:33:11Z,"['bull', 'trap', 'next']",0,$TSLA bull trap 🪤 595 next ⛔️⛔️⛔️❌❌
599996,2020-06-02T19:14:13Z,"['another', 'boring', 'day', 'like', 'agree']",0,$AAPL another boring day. Like if you agree
599997,2022-02-03T23:04:33Z,"['told', 'metaverse', 'scam']",0,$FB I told you Metaverse is a scam
599998,2020-06-12T18:23:32Z,"['mentioning', 'scam', 'breath', 'crime']",0,$NKLA Mentioning this Scam in a same breath wi...


In [ ]:
data[data['sentiment']  == 0].shape

(431285, 4)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['raw_content'], data['sentiment'], test_size=0.2, random_state=42)

In [ ]:
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

X_train: (690056,)
X_test: (172514,)
y_train: (690056,)
y_test: (172514,)


In [ ]:
MAX_LEN = 31
tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base')
def to_tokens(input_text, tokenizer):
    output = tokenizer.encode_plus(input_text, max_length=MAX_LEN, pad_to_max_length=True, return_attention_mask=False)
    return output

In [ ]:
tokenizer_output_train = X_train.apply(lambda x: to_tokens(x, tokenizer))
tokenizer_output_test = X_test.apply(lambda x: to_tokens(x, tokenizer))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
def select_field(features, field):
    return [feature[field] for feature in features]

In [ ]:
input_ids_train = np.array(select_field(tokenizer_output_train, 'input_ids'))
input_ids_test = np.array(select_field(tokenizer_output_test, 'input_ids'))

In [ ]:
input_ids = tf.keras.Input(shape=(MAX_LEN,), dtype='int32', name = "input_ids")
transformer_layer = transformers.TFRobertaModel.from_pretrained("roberta-base")
sequence_output = transformer_layer(input_ids)[0]

cls_token = sequence_output[:, 0, :]
x = tf.keras.layers.Dense(250, activation = 'relu')(cls_token)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs = input_ids, outputs = output)
model.compile(tf.keras.optimizers.Adam(learning_rate = 5e-6), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 31)]              0         
                                                                 
 tf_roberta_model (TFRoberta  TFBaseModelOutputWithPoo  124645632
 Model)                      lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             31, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                             

In [ ]:
history = model.fit(input_ids_train, y_train, validation_data=(input_ids_test, y_test), epochs=3, batch_size=32)

Epoch 1/3


21565/21565 [==============================] - 5268s 242ms/step - loss: 0.4293 - accuracy: 0.7966 - val_loss: 0.3724 - val_accuracy: 0.8342
Epoch 2/3
21565/21565 [==============================] - 5154s 239ms/step - loss: 0.3525 - accuracy: 0.8415 - val_loss: 0.3376 - val_accuracy: 0.8485
Epoch 3/3
21565/21565 [==============================] - 5186s 240ms/step - loss: 0.3185 - accuracy: 0.8597 - val_loss: 0.3337 - val_accuracy: 0.8541
